In [ ]:
import numpy as np
import pandas as pd
import cv2
train=pd.read_csv(r"C:\Users\HP\Desktop\codefest\vista\sample.csv")

In [ ]:
from pathlib import Path
img_data_path = []
for i in train['id']:
  img_data_path.append('C:/Users/HP/Desktop/codefest/vista/' +'test/'+i)

In [ ]:
cv2.imread(img_data_path[1],0)

In [ ]:
X=[]
for path in img_data_path:
    image=cv2.resize(cv2.imread(path,0),(200,100 ))
    image = np.expand_dims(image, axis=-1)
    image=image/255
    X.append(image)

In [ ]:
X=np.asarray(X)
np.save("C:/Users/HP/Desktop/codefest/vista/X2.npy",X)

In [ ]:
import os




from keras import utils
from keras.datasets import mnist # dataset
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Input, Bidirectional,Permute, concatenate, Add, Flatten, Reshape
from keras.layers import Maximum
from keras.optimizers import Adam
import keras
import cv2
import matplotlib.pyplot as plt
import keras.backend as K
import random
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import numpy as np
from keras import metrics
from keras.models import load_model

from keras.layers import Lambda, Multiply, Concatenate, UpSampling2D, Dropout, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Activation
from keras import regularizers



model_inp = Input(shape=(100,200,1))
#model_in = BatchNormalization()(model_inp)
#mul_in = MaxPooling2D(2,2)(model_inp)
#print(model_in.shape)
ip_feature = Conv2D(32, (3, 3), activation='relu', padding='same')(model_inp)
ip_feature = Conv2D(16, (3, 3), activation='relu', padding='same')(ip_feature)
ip_feature = MaxPooling2D(2,2)(ip_feature)

channel = keras.backend.int_shape(ip_feature)[3]
print(channel)

avg_pool = GlobalAveragePooling2D()(ip_feature)
avg_pool = Reshape((1,1,channel))(avg_pool)
avg_pool = Dense(channel//4)(avg_pool)
avg_pool = Dense(channel)(avg_pool)
max_pool = GlobalMaxPooling2D()(ip_feature)
max_pool = Reshape((1,1,channel))(max_pool)
max_pool = Dense(channel//4)(max_pool)
max_pool = Dense(channel)(max_pool)
channel_features = Add()([avg_pool,max_pool])
channel_fetaures = Activation('sigmoid')(channel_features)
channel_features = Multiply()([ip_feature, channel_features])
print(channel_features.shape)


avg_pool_spatial = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(channel_features)
max_pool_spatial = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(channel_features)
concat = Concatenate(axis=3)([avg_pool_spatial, max_pool_spatial])
cbam_feature = Conv2D(filters = 1,kernel_size=3,strides=1,padding='same',activation='sigmoid')(concat)
cbam_feature = Multiply()([channel_features, cbam_feature])
print(cbam_feature.shape)

add_inp_feature = Add()([ip_feature, cbam_feature])

next_conv_block = Conv2D(filters=32, kernel_size=7, padding='same', activation='relu')(add_inp_feature)
next_conv_block = AveragePooling2D((2,2))(next_conv_block)
next_conv_block = Conv2D(filters=16, kernel_size=3, padding='same', activation='relu')(next_conv_block)
next_conv_block = AveragePooling2D((2,2))(next_conv_block)
next_conv_block = Conv2D(filters=8, kernel_size=3, padding='same', activation='relu')(next_conv_block)
next_conv_block = MaxPooling2D((2,2))(next_conv_block)

x1 = Flatten()(next_conv_block)
#x1 = GlobalAveragePooling2D()(next_conv_block)

con = Dense(128)(x1)
#con = Dropout(0.25)(con)
con = Dense(64)(con)
#con = Dropout(0.25)(con)
con = Dense(32)(con)


# con = Conv2D(1, (3, 3), activation='relu', padding='same')(x1)
# con = Multiply()([con,model_in])
# con = Flatten()(con)

op = Dense(15, activation='softmax')(con)

model = Model(inputs=model_inp, outputs=op)


opt = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
filepath="C:/Users/HP/Desktop/codefest/vista/weights_attention2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.summary()

In [ ]:
X=np.load("C:/Users/HP/Desktop/codefest/vista/X2.npy")
model = load_model(r'C:\Users\HP\Downloads\M1.hdf5')

In [ ]:
prediction = model.predict(X)

In [ ]:
df_results_csv = pd.DataFrame({'id':train['id'],
                               'label':np.argmax(prediction,axis=1)})
 
df_results_csv.to_csv("C:/Users/HP/Desktop/codefest/vista/sample2.csv", index=False)

In [ ]:
import glob
cv_img = []
for img in glob.glob("C:/Users/HP/Desktop/codefest/vista/test/*.png"):
    n= cv2.imread(img,0)
    n= cv2.resize(n,(200,100))
    n=np.expand_dims(n,axis=-1)
    cv_img.append(n)

In [ ]:
cv_img=np.asarray(cv_img)

In [ ]:
cv_img.shape

In [ ]:
cv_img=cv_img/255

In [ ]:
cv_img[2]

In [ ]:
np.save("C:/Users/HP/Desktop/codefest/vista/X1.npy",cv_img)